# Digit Recognizer

This notebook presents my approach to the Kaggle Digit Recognizer competition, where the goal is to classify handwritten digits from the MNIST dataset. I implemented two neural network architectures: a Fully Connected Neural Network with Dense Layers and Dropout for regularization, and a Convolutional Neural Network (CNN) designed to leverage spatial information in the images. All hyperparameters were carefully selected through extensive experimentation and testing to achieve optimal performance. However, to maintain clarity and limit the notebook's length, the details of the testing process have not been included. Instead, this notebook focuses on showcasing the final models and their results.

#### KAGGLE COMPETITION

https://www.kaggle.com/competitions/digit-recognizer/overview

#### APPROACH

Ensemble of 2 neural networks: 
 - one fully-connected neural network (dense layers and dropout)
 - one convolutional neural network

## Fully Connected Neural Network

(all the params have been set up by testing and testing and testing...)

In [2]:
import numpy as np
from tensorflow.keras.datasets import mnist

# load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# flatten images
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# normalize in [0,1]
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# check dimensions
assert X_train.shape == (60000, 784)
assert X_test.shape == (10000, 784)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [ ]:
# create the model
from tensorflow import keras
from tensorflow.keras import layers, callbacks

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=50, # how many epochs to wait before stopping
    restore_best_weights=True,
)

model = keras.Sequential([
    layers.Dense(784, activation='relu', input_shape=[784]),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
import pandas as pd

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=256,
    epochs=500, # early stopping will stop it before 500
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=1,  # turn off training log
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
import pandas as pd

kaggle_test = pd.read_csv('./test.csv')
kaggle_test.shape

preds = model.predict(kaggle_test)
preds

In [ ]:
pred_to_save = [(x.argmax(),x.max()) for x in preds]
submission = [(i, pred_to_save[i - 1]) for i in range(1,kaggle_test.shape[0] + 1)]
submission[0]

In [8]:
import csv

data = submission

csv_file = 'fc_scores.csv'

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ImageId', 'Label', 'Score'])
    for image_id, value in data:
        label, score = value
        writer.writerow([image_id, label, score])

## Convolutional Neural Networks

(all the params have been set up by testing and testing and testing...)

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

num_classes = 10
input_shape = (28,28,1)

X_train = X_train.astype('float32') / 255 # normalize in [0,1]
X_test = X_test.astype('float32') / 255 # normalize in [0,1]

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

print(X_train.shape)
print(X_test.shape)

In [10]:
# create the model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

kernel = tf.constant([
    [-1, -1,  1],
    [-1,  3, -1],
    [ 1, -1, -1],
])

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=50, # how many epochs to wait before stopping
    restore_best_weights=True,
)

model = keras.Sequential([
    keras.Input(shape=input_shape),

    layers.Conv2D(128,
                  kernel_size=(3,3),
                  activation='relu',
                  strides=(1,1),
                  padding='valid'
                  ),

    layers.Conv2D(256, kernel_size=(3,3), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Dropout(0.2),

    layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Dropout(0.2),

    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    # layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),

    layers.Flatten(),

    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
import pandas as pd

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=500, # early stopping will stop it before 500
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=1,  # turn off training log
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
cnn_preds = model.predict(kaggle_test)

In [ ]:
pred_to_save = [(x.argmax(),x.max()) for x in cnn_preds]
submission = [(i, pred_to_save[i - 1]) for i in range(1,kaggle_test.shape[0] + 1)]
submission[0]

In [ ]:
import csv

data = submission

csv_file = 'cnn_scores.csv'

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ImageId', 'Label', 'Score'])
    for image_id, value in data:
        label, score = value
        writer.writerow([image_id, label, score])

## Ensemble

Let's use the two models to predict the label. 
If they agree on a label, then take it as true;
If they disagree, then take the 'strongest' prediction, id est the model with the higher score

In [ ]:
cnn_scores = pd.read_csv('./cnn_scores.csv')
fc_scores = pd.read_csv('./fc_scores.csv')

In [14]:
full = cnn_scores.copy()

In [15]:
full.insert(3, 'ImageId_fc', fc_scores['ImageId'])
full.insert(4, 'Label_fc', fc_scores['Label'])
full.insert(5, 'Score_fc', fc_scores['Score'])

In [16]:
final = full.copy()

In [17]:
labels = []
models = []
count = 0
for i in range(final.shape[0]):
    row = final.iloc[i]
    if (row['Label'] != row['Label_fc']):
        final_label = row['Label'] if row['Score'] >= row['Score_fc'] else row['Label_fc']
        model = 'CNN' if row['Score'] >= row['Score_fc'] else 'FC'
        labels.append(final_label)
        models.append(model)
        count += 1
    else:
        labels.append(row['Label'])
        models.append('BOTH')

print(count) # this should be 209

209


In [18]:
final.insert(6, 'Final_Label', labels)
final.insert(7, 'Best_model', models)

In [ ]:
ensemble_submission = final[['ImageId', 'Final_Label']].copy()
ensemble_submission['Final_Label'] = ensemble_submission['Final_Label'].astype(int)
ensemble_submission.rename(columns={"Final_Label": "Label"}, inplace=True)
ensemble_submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [20]:
ensemble_submission.to_csv('ensemble_submission.csv', index=False)

### SUBMISSION SCORE: 0.998